In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt


In [ ]:
dir_BLUE1    = '/Data/Output_GCB2021/202109_GCB2021_run/run_v3_ForestArea_1991-2021/'
dir_BLUE2    = '/Data/Output_GCB2021/202208_GCB2021_run/run_v4_ForestArea_1700-1730/'
dir_grids    = '/Data/grids/'
dir_for_area = '/Data/forest_area/'
dir_for_mask = '/Data/forest_masks/'
    

## Extract forest area from BLUE in 1991-2020

In [ ]:
fname = dir_BLUE1 + '202109_GCB2021_run_v3_ForestArea_1991-2021__AreasPerPFT_natural.nc'
data = xr.open_dataset(fname)

#Select PFTs 1-6 for primary (virgin) and secondary forest
vars_vir = [var for var in data.data_vars if 'area' in var and var[-1]=='v' and int(var[-3])<=6 and len(var)<11]
vars_sec = [var for var in data.data_vars if 'area' in var and var[-1]=='s' and int(var[-3])<=6 and len(var)<11]

#Calculate area of primary forest
data_vir = data[vars_vir].to_array(dim='area')
data_vir = data_vir.sum(dim='area')
data_vir = data_vir.to_dataset(name='forest_area')
data_vir.forest_area.attrs['units'] = 'ha'
data_vir.forest_area.attrs['long_name'] = 'Area of primary forest'

#Calculate area of secondary forest
data_sec = data[vars_sec].to_array(dim='area')
data_sec = data_sec.sum(dim='area')
data_sec = data_sec.to_dataset(name='forest_area')
data_sec.forest_area.attrs['units'] = 'ha'
data_sec.forest_area.attrs['long_name'] = 'Area of primary forest'

#Save in file
fname_prim = dir_for_area + 'BLUE_forest_area_primary_1991-2020.nc'
fname_sec  = dir_for_area + 'BLUE_forest_area_secondary_1991-2020.nc'
data_vir.to_netcdf(fname_prim)
data_sec.to_netcdf(fname_sec)


## Extract forest area from BLUE in 1700-1729

In [ ]:
fname = dir_BLUE2 + '202208_GCB2021_run_v4_ForestArea_1700-1730__AreasPerPFT_natural.nc'
data = xr.open_dataset(fname)

#Select PFTs 1-6 for primary (virgin) and secondary forest
vars_vir = [var for var in data.data_vars if 'area' in var and var[-1]=='v' and int(var[-3])<=6 and len(var)<11]
vars_sec = [var for var in data.data_vars if 'area' in var and var[-1]=='s' and int(var[-3])<=6 and len(var)<11]

#Calculate area of primary forest
data_vir = data[vars_vir].to_array(dim='area')
data_vir = data_vir.sum(dim='area')
data_vir = data_vir.to_dataset(name='forest_area')
data_vir.forest_area.attrs['units'] = 'ha'
data_vir.forest_area.attrs['long_name'] = 'Area of primary forest'

#Calculate area of secondary forest
data_sec = data[vars_sec].to_array(dim='area')
data_sec = data_sec.sum(dim='area')
data_sec = data_sec.to_dataset(name='forest_area')
data_sec.forest_area.attrs['units'] = 'ha'
data_sec.forest_area.attrs['long_name'] = 'Area of primary forest'

#Save in file
fname_prim = dir_for_area + 'BLUE_forest_area_primary_1701-1730.nc'
fname_sec  = dir_for_area + 'BLUE_forest_area_secondary_1701-1730.nc'
data_vir.to_netcdf(fname_prim)
data_sec.to_netcdf(fname_sec)


## Calculate average total forest fraction and regrid to forest mask

In [ ]:
#Read data
fname_prim_PD = dir_for_area + 'BLUE_forest_area_primary_1991-2020.nc'
fname_sec_PD  = dir_for_area + 'BLUE_forest_area_secondary_1991-2020.nc'
fname_prim_PI = dir_for_area + 'BLUE_forest_area_primary_1701-1730.nc'
fname_sec_PI  = dir_for_area + 'BLUE_forest_area_secondary_1701-1730.nc'
data_prim_PD = xr.open_dataset(fname_prim_PD)
data_sec_PD  = xr.open_dataset(fname_sec_PD)
data_prim_PI = xr.open_dataset(fname_prim_PI)
data_sec_PI  = xr.open_dataset(fname_sec_PI)

#Get cell area
fname = dir_BLUE1 + '202109_GCB2021_run_v3_ForestArea_1991-2021__AreasPerPFT_natural.nc'
data = xr.open_dataset(fname)
cell_area = data.cell_area

#Calculate total forest area and average over time
data_PI = data_prim_PI + data_sec_PI
data_PD = data_prim_PD + data_sec_PD
data_PI = data_PI.mean('time')
data_PD = data_PD.mean('time')

#Calculate forest fraction
data_PI = data_PI.forest_area / cell_area
data_PD = data_PD.forest_area / cell_area
data_PI = data_PI.to_dataset(name='forest_fraction')
data_PD = data_PD.to_dataset(name='forest_fraction')

#Save in file
fname_PD = dir_for_area + 'BLUE_forest_fraction_total_average-1991-2020.nc'
fname_PI = dir_for_area + 'BLUE_forest_fraction_total_average-1701-1730.nc'
if os.path.exists(fname_PD):  os.remove(fname_PD)
if os.path.exists(fname_PI):  os.remove(fname_PI)
data_PD.to_netcdf(fname_PD)
data_PI.to_netcdf(fname_PI)

#Define file names for regridding
file_grid = dir_grids + 'grid_xy_360x720-ForestMask'
fname_PD_reg = dir_for_area + 'BLUE_forest_fraction_total_average-1991-2020_regrid360x720-ForestMask.nc'
fname_PI_reg = dir_for_area + 'BLUE_forest_fraction_total_average-1701-1730_regrid360x720-ForestMask.nc'
if os.path.exists(fname_PD_reg):  os.remove(fname_PD_reg)
if os.path.exists(fname_PI_reg):  os.remove(fname_PI_reg)

#Regrid data
os.system('cdo remapcon,' + file_grid + ' ' + fname_PD + ' ' + fname_PD_reg)
os.system('cdo remapcon,' + file_grid + ' ' + fname_PI + ' ' + fname_PI_reg)


## Calculate mask for weighting SLAND of DGVMs

In [ ]:
#Define maximum values for weighting factor
max_values = [1, 2, 10, 50, 100]

#Read forest fraction
data_PD_reg = xr.open_dataset(fname_PD_reg)
data_PI_reg = xr.open_dataset(fname_PI_reg)

#Calculate weighting factor
data_frac = data_PD_reg.forest_fraction / data_PI_reg.forest_fraction

#Only allow weighting factors larger than 1 for land grid points with at least 0.1% forest cover
data_frac = xr.where(data_PI_reg.forest_fraction>0.001, data_frac, 1)
data_frac = xr.where(data_PD_reg.forest_fraction!=0, data_frac, 0)

#Replace NaNs by 0
data_frac = data_frac.fillna(0)

#Convert to data set
data_frac = data_frac.to_dataset(name='w_FF')
    
#Save weight in file
fname_weight = dir_for_mask + 'Weights_ForestFraction_BLUE_regrid360x720-ForestMask.nc'
if os.path.exists(fname_weight):  os.remove(fname_weight)
data_frac.to_netcdf(fname_weight)
